In [3]:
pip install lancedb

Note: you may need to restart the kernel to use updated packages.


In [4]:
import lancedb

In [5]:
uri = 'data/new1'

In [6]:
db = lancedb.connect(uri)

In [7]:
data =[ 
    
    {'vector' : [1,1], 'item' : 'food', 'price':150},
    {'vector' : [2,2], 'item' : 'oil', 'price':20},
    {'vector' : [3,3], 'item' : 'milk', 'price':100}
    
]

In [31]:
data[1]['vector']

[2, 2]

In [9]:
tbl = db.create_table('Table1', data = data)

In [10]:
import pyarrow as pa

In [34]:
##schema = pa.schema([pa.field('vector', pa.list_(pa.float32(), list_size =2 ))])
schema = pa.schema([
    pa.field('vector', pa.list_(pa.float32(), list_size=2)),
    pa.field('item', pa.string()),
    pa.field('price', pa.float32())  # or pa.int32() if price is always integer
])


In [40]:
tbl2 = db.create_table('table2' , schema=schema)
##db.drop_table("table2")

In [41]:
tbl2 = db.open_table('table2')

In [42]:
print(db.list_tables())

tables=['Table1', 'table2'] page_token=None


In [43]:
tbl2.add(data)

AddResult(version=2)

In [45]:
tbl2

LanceTable(name='table2', version=2, _conn=LanceDBConnection(uri='C:\\Users\\f.zarei\\VDB\\data\\new1'))

In [56]:
tbl.search([5,5]).limit(1).to_pandas()

,vector,item,price,_distance
0,"[1, 1]",food,150,2.0


In [58]:
tbl.search('milk').limit(1).to_pandas()

RuntimeError: lance error: Invalid user input: Cannot perform full text search unless an INVERTED index has been created on at least one column, C:\Users\runneradmin\.cargo\registry\src\index.crates.io-1949cf8c6b5b557f\lance-index-1.0.1\src\scalar\inverted\query.rs:764:25

In [63]:
pip install tantivy

Note: you may need to restart the kernel to use updated packages.


In [64]:
tbl.search('milk').limit(1).to_pandas()

RuntimeError: lance error: Invalid user input: Cannot perform full text search unless an INVERTED index has been created on at least one column, C:\Users\runneradmin\.cargo\registry\src\index.crates.io-1949cf8c6b5b557f\lance-index-1.0.1\src\scalar\inverted\query.rs:764:25

In [66]:
tbl.create_fts_index("item")

In [67]:
tbl

LanceTable(name='Table1', version=2, _conn=LanceDBConnection(uri='C:\\Users\\f.zarei\\VDB\\data\\new1'))

In [68]:
tbl.search('milk').limit(2).to_pandas()

,vector,item,price,_score
0,"[3, 3]",milk,100,0.980829


In [71]:
tbl.search('item == milk').limit(2).to_pandas()

,vector,item,price,_score
0,"[3, 3]",milk,100,0.980829


In [73]:
tbl.delete('item == "milk"')

DeleteResult(version=3)

In [74]:
tbl.search([5,5]).limit(2).to_pandas()

,vector,item,price,_distance
0,"[1, 1]",food,150,2.0
1,"[2, 2]",oil,20,6.0


In [75]:
db.drop_table('table2')

In [76]:
data2 = [
    
    {'vector': [1,5], 'item': 'rice', 'price' : 1000}
    
]

In [77]:
tbl.add(data2)

AddResult(version=4)

In [78]:
tbl.search([1,5]).limit(3).to_pandas()

,vector,item,price,_distance
0,"[1, 5]",rice,1000,0.0
1,"[1, 1]",food,150,1.0
2,"[2, 2]",oil,20,5.0


In [79]:
tbl.delete( 'price = 1000')

DeleteResult(version=5)

In [80]:
tbl.search([1,5]).limit(3).to_pandas()

,vector,item,price,_distance
0,"[1, 1]",food,150,1.0
1,"[2, 2]",oil,20,5.0


In [81]:
tbl.update(where="price = 150", values={"vector": [10, 10]})

UpdateResult(rows_updated=1, version=6)

In [82]:
tbl.search([1,5]).limit(3).to_pandas()

,vector,item,price,_distance
0,"[2, 2]",oil,20,5.0
1,"[10, 10]",food,150,7.0
